In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import sqlite3

In [18]:
Veri_Seti3_201701_In = pd.read_csv('./Dataset 3/Dataset 3_201701_In.csv')

In [19]:
dataset = pd.DataFrame(Veri_Seti3_201701_In)
dataset.head()

,CALLER_ID,TIMESTAMP,ID,CITY_ID
0,1100144895,02-01-2017 12:52,568,7
1,1100144895,02-01-2017 20:40,568,7
2,1100144895,02-01-2017 21:41,568,7
3,1100144895,03-01-2017 14:57,828,7
4,1100144895,05-01-2017 10:01,568,7


## Extract information from January dataset

In [20]:
print ('there are {} calls recorded in this dataset'.format(len(dataset)))

there are 998625 calls recorded in this dataset


### Extract timestamp

In [23]:
%%time
dataset['TIMESTAMP'] = pd.to_datetime(dataset['TIMESTAMP'], format = "%d-%m-%Y %H:%M")

CPU times: user 17.5 ms, sys: 41 µs, total: 17.5 ms
Wall time: 7.26 ms


### Extract called ID

In [30]:
dataset['CALLER_ID'] = dataset['CALLER_ID'].astype(str)

### Extract prefix

In [47]:
#get dataset with only refugee calls
refugee_calls = dataset[dataset['CALLER_ID'].str.startswith('1')]
refugee_calls.head()

,CALLER_ID,TIMESTAMP,ID,CITY_ID
0,1100144895,2017-01-02 12:52:00,568,7
1,1100144895,2017-01-02 20:40:00,568,7
2,1100144895,2017-01-02 21:41:00,568,7
3,1100144895,2017-01-03 14:57:00,828,7
4,1100144895,2017-01-05 10:01:00,568,7


In [50]:
#get dataset with only nonrefugee calls
nonrefugee_calls = dataset[dataset['CALLER_ID'].str.startswith('2')]
nonrefugee_calls.head()

,CALLER_ID,TIMESTAMP,ID,CITY_ID
239897,2100034000,2017-01-01 00:30:00,949,35
239898,2100034000,2017-01-01 00:55:00,949,35
239899,2100034000,2017-01-01 00:58:00,949,35
239900,2100034000,2017-01-01 01:07:00,949,35
239901,2100034000,2017-01-01 01:26:00,949,35


In [41]:
refugees_ids = refugee_calls.CALLER_ID.unique()

array(['1100144895', '1100239022', '1100256098', ..., '1999665531',
       '1999760991', '1999796150'], dtype=object)

In [42]:
print ('there are {} refugee calls in this dataset'.format(len(refugee_calls)))
print ('there are {} different refugees in this dataset'.format(len(refugees_ids)))

there are 239897 refugee calls in this dataset
there are 5525 different refugees in this dataset


## Analyse data

In [133]:
ref_evening = refugee_calls[(refugee_calls.TIMESTAMP.dt.hour >=18) | (refugee_calls.TIMESTAMP.dt.hour <=7)]
ref_evening = ref_evening.set_index(['CALLER_ID', 'CITY_ID'], drop = False)
ref_evening

CALLER_ID           TIMESTAMP   ID  CITY_ID
CALLER_ID  CITY_ID                                              
1100144895 7        1100144895 2017-01-02 20:40:00  568        7
           7        1100144895 2017-01-02 21:41:00  568        7
           7        1100144895 2017-01-06 21:38:00  568        7
           7        1100144895 2017-01-06 21:44:00  568        7
           7        1100144895 2017-01-06 21:58:00  568        7
           7        1100144895 2017-01-08 22:31:00  568        7
           7        1100144895 2017-01-11 21:02:00  568        7
           7        1100144895 2017-01-11 22:26:00  568        7
           7        1100144895 2017-01-13 21:04:00  568        7
1100239022 6        1100239022 2017-01-05 18:29:00  528        6
           6        1100239022 2017-01-05 18:58:00  528        6
           6        1100239022 2017-01-05 19:34:00  689        6
           6        1100239022 2017-01-05 19:56:00  689        6
           6        1100239022 2017-01-06 18:43:00  528        6
           6        1100239022 2017-01-06 18:44:00  528        6
           6        1100239022 2017-01-06 19:09:00  528        6
           6        1100239022 2017-01-06 20:02:00  528        6
           6        1100239022 2017-01-07 19:23:00  528        6
           6        1100239022 2017-01-07 20:25:00  528        6
           6        1100239022 2017-01-08 19:21:00  528        6
           6        1100239022 2017-01-09 19:57:00  528        6
           6        1100239022 2017-01-09 20:28:00  528        6
           6        1100239022 2017-01-11 18:37:00  528        6
           6        1100239022 2017-01-11 19:33:00  528        6
           6        1100239022 2017-01-11 19:34:00  528        6
           6        1100239022 2017-01-12 18:35:00  528        6
           6        1100239022 2017-01-12 19:19:00  528        6
           6        1100239022 2017-01-12 20:43:00  528        6
           6        1100239022 2017-01-16 18:50:00  528        6
           6        1100239022 2017-01-16 19:31:00  528        6
...                        ...                 ...  ...      ...
1999665531 34       1999665531 2017-01-12 19:13:00  386       34
           34       1999665531 2017-01-12 19:33:00  386       34
           34       1999665531 2017-01-12 19:38:00  386       34
           34       1999665531 2017-01-14 19:30:00  386       34
           34       1999665531 2017-01-15 21:56:00  386       34
           34       1999665531 2017-01-16 19:10:00  386       34
1999760991 34       1999760991 2017-01-01 18:23:00  401       34
           34       1999760991 2017-01-02 18:02:00  537       34
           34       1999760991 2017-01-02 18:08:00  537       34
           34       1999760991 2017-01-02 18:22:00  537       34
           34       1999760991 2017-01-02 18:30:00  537       34
           34       1999760991 2017-01-02 19:42:00  537       34
           34       1999760991 2017-01-02 20:43:00  537       34
           34       1999760991 2017-01-03 19:42:00  537       34
           34       1999760991 2017-01-05 18:46:00  537       34
           34       1999760991 2017-01-05 23:05:00  401       34
           34       1999760991 2017-01-06 23:23:00  401       34
           34       1999760991 2017-01-06 23:30:00  401       34
           34       1999760991 2017-01-07 21:53:00  401       34
           34       1999760991 2017-01-08 22:19:00  401       34
           34       1999760991 2017-01-11 18:10:00  512       34
           34       1999760991 2017-01-13 18:00:00  537       34
           34       1999760991 2017-01-13 18:01:00  537       34
           34       1999760991 2017-01-13 19:01:00  537       34
           34       1999760991 2017-01-13 20:25:00  401       34
           34       1999760991 2017-01-14 20:22:00  401       34
           34       1999760991 2017-01-15 18:28:00  401       34
           34       1999760991 2017-01-15 21:01:00  401       34
           34       1999760991 2017-01-15 21:25:00  401 

In [147]:
mobile = ref_evening.groupby('CALLER_ID')['CITY_ID'].apply(lambda g: len(g.unique())>1)

/home/arnau/miniconda3/envs/py/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'CALLER_ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


In [148]:
print ('there are {} callers out ot 50,000 refugees who moved cities'.format(len(mobile)))

there are 5132 callers out ot 50,000 refugees who moved cities


Calculated cities moved between (i.e. were in 'residence' for for more that 4 nights) 

In [ ]:
#for i in mobile:
#    moved = [evening_location[i]['cities'][0]]
#    date_moved =  date[0]
#    for j in range(1,len(evening_location[i]['cities'])):
#        if evening_location[i]['cities'][j] != evening_location[i]['cities'][j-1]:
#            diff = np.timedelta64(evening_location[i]['dates'][j-1]-date_moved)
#            diff = int(diff.astype('timedelta64[D]') / np.timedelta64(1, 'D'))
#            if diff >=4:
#                moved.append(evening_location[i]['cities'][j-1])
#        date_moved = evening_location[i]['dates'][j]
#    evening_location[i]['moved'] = moved

In [ ]:
for i in mobile:
    moved = []
    moved.append(evening_location[i]['cities'][0])
    counter = 0
    for j in range(1,len(evening_location[i]['cities'])):
        if evening_location[i]['cities'][j] == evening_location[i]['cities'][j-1]:
            counter += 1
        else:
            counter = 0
        if counter == 4 and evening_location[i]['cities'][j] != moved[-1]:
            moved.append(evening_location[i]['cities'][j])
    evening_location[i]['moved'] = moved

In [ ]:
low_mobility = []
medium_mobility = []
high_mobility = []
for i in mobile:
    if len(evening_location[i]['moved']) <=2:
        low_mobility.append(i)
    elif 2 < len(evening_location[i]['moved']) <=5:
        medium_mobility.append(i)
    else:
        high_mobility.append(i)

In [ ]:
print ('There are {} refugees classed as low mobility'.format(len(low_mobility)))
print ('There are {} refugees classed as medium mobility'.format(len(medium_mobility)))
print ('There are {} refugees classed as high mobility'.format(len(high_mobility)))

### Compare against non-refugee population

In [ ]:
non_to_analyse = np.unique(callerID[prefix==1])

In [ ]:
non_h1 = callerID[timestamp.hour >= 18]
non_h2 = callerID[timestamp.hour <= 7]
non_h = np.append(non_h1,non_h2)
non_overlap = np.intersect1d(non_h,non_to_analyse)

In [ ]:
non_evening_location = {}
for i in non_overlap:
    non_evening_location[str(i)] = {
        'cities':[],
        'dates':[]
    }

In [ ]:
##bottleneck - could parallelise better
for i in non_overlap:
    location =  np.where(callerID == i)
    cities = cityID[location[0]]
    non_evening_location[i]['cities'] = cities
    dates = date[location[0]]
    non_evening_location[i]['dates'] = dates

In [ ]:
with open("non_evening_location.txt", "wb") as fp:   #Pickling
    pickle.dump(non_evening_location, fp)

In [ ]:
with open("non_evening_location.txt", "rb") as fp:   # Unpickling
    non_evening_location = pickle.load(fp)

In [ ]:
non_mobile = [] #find the refugees who spend their time in more that one city in the evening over the month
for i in non_evening_location:
    unique, counts = np.unique(non_evening_location[i]['cities'], return_counts=True)
    if len(unique) > 1:
        non_mobile.append(i)

In [ ]:
print ('there are {} callers out ot 50,000 non-refugees who moved cities'.format(len(non_mobile)))

In [ ]:
print ('there are {} times as many refugees who moved cities as non-refugees'.format(
    round(float(len(mobile))/float(len(non_mobile)),2)))